In [1]:
from ResNet_50 import ResNet50
import torch
from cifar_data import CifarDataset
import torch.nn as nn
from torch.nn.init import _calculate_fan_in_and_fan_out, _no_grad_uniform_
import math
import numpy as np


In [2]:
def xavier_uniform_M(tensor, gain: float = 1., fac = 3.):
    r"""Fills the input `Tensor` with values according to the method
        described in `Understanding the difficulty of training deep feedforward
        neural networks` - Glorot, X. & Bengio, Y. (2010), using a uniform
        distribution. The resulting tensor will have values sampled from
        :math:`\mathcal{U}(-a, a)` where

        .. math::
            a = \text{gain} \times \sqrt{\frac{6}{\text{fan\_in} + \text{fan\_out}}}

        Also known as Glorot initialization.

        Args:
            tensor: an n-dimensional `torch.Tensor`
            gain: an optional scaling factor

        Examples:
            >>> w = torch.empty(3, 5)
            >>> nn.init.xavier_uniform_(w, gain=nn.init.calculate_gain('relu'))
        """
    fan_in, fan_out = _calculate_fan_in_and_fan_out(tensor)
    std = gain * math.sqrt(2.0 / float(fan_in + fan_out))
    a = math.sqrt(fac) * std  # Calculate uniform bounds from standard deviation

    return _no_grad_uniform_(tensor, -a, a)

def weights_init(m, method):
    """
    Function for filters initialization
    * function uses method from PyTorch to initialize weights
    @ m - model
    @ method - method to initialize weights
    TODO: add custom method to initialize weights
    """
    with torch.no_grad():
        if isinstance(m, nn.Conv2d):
            if method == 'orthogonal':
                torch.nn.init.orthogonal_(m.weight)  # type: ignore
            elif method == 'xavier_uniform':
                torch.nn.init.xavier_uniform_(m.weight, gain=1.0)
            elif method == 'xavier_uniform_M_2':
                    self.xavier_uniform_M(m.weight, gain=1.0, fac=1.)
            elif method == 'xavier_uniform_M_10':
                self.xavier_uniform_M(m.weight, gain=1.0, fac=5.)
            elif method == 'xavier_uniform_M_14':
                self.xavier_uniform_M(m.weight, gain=1.0, fac=7.)
            elif method == 'xavier_uniform_M_20':
                self.xavier_uniform_M(m.weight, gain=1.0, fac=10.)
            elif method == 'xavier_uniform_M_1':
                self.xavier_uniform_M(m.weight, gain=1.0, fac=0.5)


In [3]:
model = ResNet50(10)
model.apply(lambda m: weights_init(m, "xavier_uniform"))
is_cuda = torch.cuda.is_available()
if is_cuda:
    model = model.cuda()
    print("GPU is available")
else:
    torch.device("cpu")
    print("GPU not available, CPU used")

optimizer = torch.optim.SGD(model.parameters(), lr=0.001,  momentum=0.9)
data = CifarDataset()
criterion = nn.CrossEntropyLoss()


GPU is available
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [4]:

def test(dataset, model, criterion):
    total = 0.
    predicted = 0.
    loss_t = 0
    with torch.no_grad():
        for data, labels in dataset.testloader:
            labels = labels.cuda()
            data = data.cuda()
            out = model(data)
            output = torch.argmax(out, dim=1)
            
            total += labels.shape[0]
            predicted += torch.sum(output == labels).cpu().item()
            
            loss = criterion(out, labels)
            loss_t += loss.cpu().item()
            

    return predicted/total*100  # type:ignore


In [5]:
EPOCHS = 200
for epoch in range(EPOCHS):
    losses = []
    running_loss = 0
    for i, inp in enumerate(data.trainloader):
        inputs, labels = inp
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 0 and i > 0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss / 100)
            running_loss = 0.0
    acc = test(data, model, criterion)
    print("acc: {:.2f} %".format(acc))
    avg_loss = sum(losses)/len(losses)

print('Training Done')


Loss [1, 100](epoch, minibatch):  2.1888458633422854
Loss [1, 200](epoch, minibatch):  2.0208187007904055
Loss [1, 300](epoch, minibatch):  1.9455267333984374
Loss [1, 400](epoch, minibatch):  1.9043874430656433
acc: 11.28 %
Loss [2, 100](epoch, minibatch):  1.8088793027400971
Loss [2, 200](epoch, minibatch):  1.7577320384979247
Loss [2, 300](epoch, minibatch):  1.7103248071670532
Loss [2, 400](epoch, minibatch):  1.7024067270755767
